The object of this program is loop through the fracfocus CSV files and add them to a SQLite database.

First we bring in the modules we need for this project

In [ ]:
import sqlite3                     #Bring in Sqlite for the database

from os import listdir             #Bring in OS to read all the files
from os.path import isfile, join  

import pandas as pd                #Bring in Pandas to work wiht the data

Download the latest fracfocus data from http://fracfocus.org/data-download.  Download the csv files and save the unzipped files to your computer.  Then update 'mypath' below to that location.

The program then loops throught the file names in the folder then saves them to 2 lists.

In [ ]:
mypath = "C:/Users/BWeaver/Google Drive/FracFocus/FracFocus"  #make sure the use back slashes or double forward slashes '\\'
mypath = '/Users/brianweaver/Google Drive/FracFocus'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]   #find files in the directory

FracFocusRegistry_files = list()
registryupload_files = list()

#loop through list of files and add them to the two lists
for file in onlyfiles:
    if file[:17] == 'FracFocusRegistry':
        FracFocusRegistry_files.append(file)
    elif file[:14] == 'registryupload':
        registryupload_files.append(file)

The program will then build the SQLite database if the file isn't there.  If the file is there then it will delete the FracFocusRegistry and registryupload tables.  

Then rebuild them with all the correct headers.  This was taken from the headers of the csv files

In [ ]:
conn = sqlite3.connect(mypath + '/FracFocus.sqlite')
cur = conn.cursor()

cur.execute('''
DROP TABLE IF EXISTS FracFocusRegistry;
''')
cur.execute('''
CREATE TABLE FracFocusRegistry (
    id                        INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    UploadKey                 TEXT,
    JobStartDate              NUMERIC,
    JobEndDate                NUMERIC,
    APINumber                 TEXT,
    StateNumber               NUMERIC,
    CountyNumber              NUMERIC,
    OperatorName              TEXT,
    WellName                  TEXT,
    Latitude                  TEXT,
    Longitude                 TEXT,
    Projection                TEXT,
    TVD                       TEXT,
    TotalBaseWaterVolume      TEXT,
    TotalBaseNonWaterVolume   TEXT,
    StateName                 TEXT,
    CountyName                TEXT,
    FFVersion                 TEXT,
    FederalWell               TEXT,
    IndianWell                TEXT,
    Source                    TEXT,
    DTMOD                     TEXT,
    PurposeKey                TEXT,
    TradeName                 TEXT,
    Supplier                  TEXT,
    Purpose                   TEXT,
    SystemApproach            TEXT,
    IsWater                   TEXT,
    PurposePercentHFJob       TEXT,
    PurposeIngredientMSDS     TEXT,
    IngredientKey             TEXT,
    IngredientName            TEXT,
    CASNumber                 TEXT,
    PercentHighAdditive       TEXT,
    PercentHFJob              TEXT,
    IngredientComment         TEXT,
    IngredientMSDS            TEXT,
    MassIngredient            TEXT,
    ClaimantCompany           TEXT,
    DisclosureKey             TEXT
    
);
''')

cur.execute('''
DROP TABLE IF EXISTS registryupload;
''')

cur.execute('''
CREATE TABLE registryupload (
    id                        INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    pKey                      TEXT,
    JobStartDate              NUMERIC,
    JobEndDate                NUMERIC,
    APINumber                 TEXT,
    StateNumber               NUMERIC,
    CountyNumber              NUMERIC,
    OperatorName              TEXT,
    WellName                  TEXT,
    Latitude                  TEXT,
    Longitude                 TEXT,
    Projection                TEXT,
    TVD                       TEXT,
    TotalBaseWaterVolume      TEXT,
    TotalBaseNonWaterVolume   TEXT,
    StateName                 TEXT,
    CountyName                TEXT,
    FFVersion                 TEXT,
    FederalWell               TEXT,
    IndianWell                TEXT,
    Source                    TEXT,
    DTMOD                     TEXT
);
''')


Now that the database is built we'll start loading it with data from the csv files.  Starting wiht the FracFocusRegistry.

In [ ]:
for file in FracFocusRegistry_files:
    print(file)
    df = pd.read_csv(mypath + '/' + file)
    df.to_sql('FracFocusRegistry', conn, if_exists='append', index=False)
print('Complete!')

Next we upload the registryupload

In [ ]:
for file in registryupload_files:
    print(file)
    df = pd.read_csv(mypath + '/' + file)
    df.to_sql('registryupload', conn, if_exists='append', index=False)
print('Complete!')

Next close the connections

In [ ]:
cur.close()
conn.close()

Now the Database is setup